In [2]:
!pip install fastapi uvicorn sentence-transformers chromadb python-multipart PyPDF2 docx2txt


  Using cached fastapi-0.115.4-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached chromadb-0.5.18-py3-none-any.whl.metadata (6.8 kB)
  Using cached python_multipart-0.0.17-py3-none-any.whl.metadata (1.8 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232

In [4]:
!pip install pyngrok

In [5]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from sentence_transformers import SentenceTransformer
import chromadb
import os
import uuid
from pyngrok import ngrok

app = FastAPI()


In [6]:

client = chromadb.Client()
collection = client.create_collection("document_embeddings")


In [7]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from PyPDF2 import PdfReader
import docx2txt


def extract_text(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.pdf':
        reader = PdfReader(file_path)
        text = ''.join([page.extract_text() for page in reader.pages])
    elif ext in ['.doc', '.docx']:
        text = docx2txt.process(file_path)
    elif ext == '.txt':
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    else:
        raise ValueError("Unsupported file type")
    return text

def generate_embedding(text: str):
    return model.encode([text])[0]


In [9]:
@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    try:
        file_id = str(uuid.uuid4())
        file_path = f"/content/{file_id}_{file.filename}"

        with open(file_path, "wb") as f:
            f.write(await file.read())

        text = extract_text(file_path)
        embedding = generate_embedding(text)

        collection.add(
            documents=[text],
            metadatas=[{"filename": file.filename}],
            ids=[file_id]
        )

        return JSONResponse(content={"message": "File processed successfully", "file_id": file_id})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [14]:
@app.post("/query/")
async def query_documents(query: str):
    try:
        query_embedding = generate_embedding(query)

        results = collection.query(query_embeddings=[query_embedding], n_results=5)

        return JSONResponse(content={"results": results})
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [38]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

!nohup uvicorn main:app --host 0.0.0.0 --port 8000 &


nohup: appending output to 'nohup.out'


In [50]:
!ps -aux | grep uvicorn


root       14794  0.0  0.0   6484  2340 ?        S    19:18   0:00 grep uvicorn


In [51]:
from pyngrok import ngrok

tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")


Public URL: NgrokTunnel: "https://58e3-34-134-10-66.ngrok-free.app" -> "http://localhost:8000"


In [42]:
from google.colab import files
uploaded = files.upload()


Saving Pranjal_Tripathi_Resume.pdf to Pranjal_Tripathi_Resume.pdf


In [53]:
import requests

upload_url = f"{public_url.public_url}/upload/" 

files = {'file': open('/content/Pranjal_Tripathi_Resume.pdf', 'rb')}

response = requests.post(upload_url, files=files)
print("Upload Response:", response.text) 

try:
    print("Upload Response (JSON):", response.json())
except requests.exceptions.JSONDecodeError:
    print("The server response was not valid JSON.")

Upload Response: <!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mon

In [48]:
import requests

upload_url = f"{public_url.public_url}/upload/"

files = {'file': open('/content/Pranjal_Tripathi_Resume.pdf', 'rb')}

response = requests.post(upload_url, files=files)

print("Response Status Code:", response.status_code)

print("Response Content:", response.text)

try:
    print("Upload Response (JSON):", response.json())
except requests.exceptions.JSONDecodeError:
    print("The server response was not valid JSON.")

Response Status Code: 400
Response Content: <!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.co